# Motor de Stress Testing en Python
## Escenarios de Estres y Cambios de Regimen de Mercado

**Practica B2-2 - Modulo de Gestion de Riesgos**

| | |
|---|---|
| **Autor** | Raul Rodriguez |
| **Fecha** | Febrero 2026 |
| **Periodo de datos** | 01-01-2006 a la fecha |

---

### Descripcion del problema

Los modelos de riesgo tradicionales fallaron en 2022: acciones y bonos cayeron simultaneamente, 
y los enfoques basados en la normalidad y correlaciones estaticas no reflejaron el deterioro real del riesgo.

Este notebook implementa un motor de stress testing capaz de:
1. Detectar regimenes de mercado (calma vs crisis) usando Hidden Markov Models
2. Modelar la estructura de dependencia entre activos mediante copulas
3. Simular escenarios de estres con Monte Carlo
4. Cuantificar perdidas extremas: VaR 99% y Expected Shortfall (CVaR)

### Cartera analizada

Cartera equiponderada (sin rebalanceo) compuesta por 18 activos:
- **Acciones:** AAPL, AMZN, BAC, BRK-B, CVX, ENPH, GME, GOOGL, JNJ, JPM, MSFT, NVDA, PG, XOM
- **Oro:** GLD
- **Renta Fija:** Bono Treasury 10Y, Bono Treasury 2Y, HYG (High Yield)


---
## 1. Configuracion y carga de librerias

In [ ]:
# ==============================================================================
# CONFIGURACION GENERAL
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Manipulacion de datos
import numpy as np
import pandas as pd
from datetime import datetime

# Descarga de datos de mercado
import yfinance as yf

# Estadistica
from scipy import stats
from scipy.stats import norm, skew, kurtosis

# Hidden Markov Models
from hmmlearn.hmm import GaussianHMM

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Configuracion de graficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

# Paleta de colores
COLOR_CALMA = '#2E86AB'   # Azul
COLOR_CRISIS = '#E94F37'  # Rojo

# Semilla para reproducibilidad
SEED = 42
np.random.seed(SEED)

# Parametros del analisis
START_DATE = '2006-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
N_SIMULATIONS = 10_000    # Numero de trayectorias Monte Carlo
HORIZON_DAYS = 126        # Horizonte de simulacion (6 meses)

print(f"Configuracion cargada correctamente")
print(f"Periodo de datos: {START_DATE} a {END_DATE}")
print(f"Simulaciones Monte Carlo: {N_SIMULATIONS:,} trayectorias")
print(f"Horizonte: {HORIZON_DAYS} dias de trading (~6 meses)")


In [ ]:
# ==============================================================================
# DEFINICION DEL UNIVERSO DE ACTIVOS
# ==============================================================================

# Activos de la cartera segun enunciado
PORTFOLIO_TICKERS = {
    'AAPL': 'Apple',
    'AMZN': 'Amazon',
    'BAC': 'Bank of America',
    'BRK-B': 'Berkshire Hathaway',
    'CVX': 'Chevron',
    'ENPH': 'Enphase Energy',
    'GLD': 'SPDR Gold Trust',
    'GME': 'GameStop',
    'GOOGL': 'Alphabet',
    'JNJ': 'Johnson & Johnson',
    'JPM': 'JPMorgan',
    'MSFT': 'Microsoft',
    'NVDA': 'NVIDIA',
    'PG': 'Procter & Gamble',
    'XOM': 'ExxonMobil',
    'IEF': 'Treasury 10Y (ETF proxy)',
    'SHY': 'Treasury 2Y (ETF proxy)',
    'HYG': 'High Yield Corporate Bonds'
}

# Indice de mercado para deteccion de regimenes
MARKET_TICKER = '^GSPC'  # S&P 500

TICKERS = list(PORTFOLIO_TICKERS.keys())

print(f"Universo de {len(TICKERS)} activos definido")


---
## 2. Obtencion y preparacion de datos

In [ ]:
# ==============================================================================
# DESCARGA DE DATOS HISTORICOS
# ==============================================================================

def download_market_data(tickers, start_date, end_date):
    """
    Descarga precios de cierre ajustados desde Yahoo Finance.
    
    Parametros:
        tickers: lista de simbolos a descargar
        start_date: fecha de inicio (formato 'YYYY-MM-DD')
        end_date: fecha de fin
        
    Retorna:
        DataFrame con precios de cierre ajustados
    """
    all_tickers = tickers + [MARKET_TICKER]
    
    print(f"Descargando datos para {len(all_tickers)} activos...")
    
    data = yf.download(
        all_tickers,
        start=start_date,
        end=end_date,
        progress=True,
        auto_adjust=True
    )['Close']
    
    # Limpiar MultiIndex si existe
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(0)
    
    print(f"Datos descargados: {len(data)} observaciones")
    print(f"Rango: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")
    
    return data

# Ejecutar descarga
prices = download_market_data(TICKERS, START_DATE, END_DATE)
prices.head()


In [ ]:
# ==============================================================================
# CALCULO DE RETORNOS LOGARITMICOS
# ==============================================================================

def calculate_log_returns(prices):
    """
    Calcula retornos logaritmicos diarios.
    
    Los retornos logaritmicos se utilizan porque:
    - Son aditivos en el tiempo
    - Tienen una distribucion mas simetrica que los retornos simples
    - Facilitan el calculo de estadisticos
    
    Formula: r_t = ln(P_t / P_{t-1})
    """
    log_returns = np.log(prices / prices.shift(1))
    return log_returns.dropna()

returns = calculate_log_returns(prices)

# Estadisticas descriptivas
print("Estadisticas de retornos diarios:")
print("-" * 50)
summary = pd.DataFrame({
    'Media (%)': returns.mean() * 100,
    'Vol diaria (%)': returns.std() * 100,
    'Vol anual (%)': returns.std() * np.sqrt(252) * 100,
    'Min (%)': returns.min() * 100,
    'Max (%)': returns.max() * 100
}).round(3)
display(summary)


---
## 3. Deteccion de regimenes de mercado (Hidden Markov Model)

El objetivo es identificar, para cada dia, si el mercado se encuentra en un estado de **calma** 
o de **crisis**. Para ello se ajusta un modelo HMM (Hidden Markov Model) gaussiano de 2 estados 
sobre los retornos del S&P 500.

El HMM asume que existe un estado oculto (no observable directamente) que gobierna el comportamiento 
de los retornos. El modelo estima:
- Las probabilidades de transicion entre estados
- Los parametros de cada estado (media y varianza de los retornos)


In [ ]:
# ==============================================================================
# AJUSTE DEL MODELO HMM DE 2 ESTADOS
# ==============================================================================

def fit_regime_model(returns, n_states=2):
    """
    Ajusta un Hidden Markov Model gaussiano para detectar regimenes de mercado.
    
    Parametros:
        returns: DataFrame con retornos (debe incluir el ticker del mercado)
        n_states: numero de estados ocultos (por defecto 2: calma y crisis)
        
    Retorna:
        model: modelo HMM ajustado
        states: array con el estado asignado a cada observacion
    """
    # Usar retornos del mercado para la deteccion de regimenes
    market_returns = returns[MARKET_TICKER].values.reshape(-1, 1)
    
    # Configurar y ajustar el modelo
    model = GaussianHMM(
        n_components=n_states,
        covariance_type='full',
        n_iter=100,
        random_state=SEED
    )
    model.fit(market_returns)
    
    # Obtener secuencia de estados
    states = model.predict(market_returns)
    
    # Identificar cual estado es crisis (mayor volatilidad)
    vol_state_0 = market_returns[states == 0].std()
    vol_state_1 = market_returns[states == 1].std()
    
    # Reordenar para que 0=calma, 1=crisis
    if vol_state_0 > vol_state_1:
        states = 1 - states
        model.means_ = model.means_[::-1]
        model.covars_ = model.covars_[::-1]
        model.transmat_ = model.transmat_[::-1, ::-1]
    
    vol_calma = market_returns[states == 0].std()
    vol_crisis = market_returns[states == 1].std()
    
    print("Modelo HMM ajustado correctamente")
    print(f"  - Estado 0 (Calma):  volatilidad = {vol_calma:.4f}")
    print(f"  - Estado 1 (Crisis): volatilidad = {vol_crisis:.4f}")
    print(f"  - Ratio de volatilidades: {vol_crisis/vol_calma:.2f}x")
    
    return model, states

hmm_model, states = fit_regime_model(returns)
returns['State'] = states


In [ ]:
# ==============================================================================
# MATRIZ DE TRANSICION
# ==============================================================================

def analyze_transition_matrix(model):
    """
    Analiza la matriz de transicion del HMM.
    
    La matriz de transicion indica las probabilidades de pasar de un estado a otro:
    - transmat[i,j] = P(estado en t+1 = j | estado en t = i)
    """
    transmat = model.transmat_
    
    trans_df = pd.DataFrame(
        transmat,
        index=['Desde: Calma', 'Desde: Crisis'],
        columns=['Hacia: Calma', 'Hacia: Crisis']
    )
    
    print("MATRIZ DE TRANSICION")
    print("=" * 50)
    print("Interpretacion: P(estado manana | estado hoy)")
    print()
    display(trans_df.round(4))
    
    # Duracion esperada de cada estado
    # E[duracion] = 1 / P(salir del estado)
    dur_calma = 1 / transmat[0, 1] if transmat[0, 1] > 0 else float('inf')
    dur_crisis = 1 / transmat[1, 0] if transmat[1, 0] > 0 else float('inf')
    
    print()
    print("Interpretacion economica:")
    print(f"  - Duracion esperada de periodos de calma:  {dur_calma:.0f} dias ({dur_calma/252:.1f} anios)")
    print(f"  - Duracion esperada de periodos de crisis: {dur_crisis:.0f} dias ({dur_crisis/21:.1f} meses)")
    
    return trans_df

transition_matrix = analyze_transition_matrix(hmm_model)


In [ ]:
# ==============================================================================
# VISUALIZACION: S&P 500 CON REGIMENES DETECTADOS
# ==============================================================================

def plot_regimes(prices, states):
    """
    Grafico del S&P 500 coloreado segun el regimen detectado.
    """
    fig, axes = plt.subplots(3, 1, figsize=(16, 10), 
                             gridspec_kw={'height_ratios': [3, 1, 1]})
    
    price_data = prices[MARKET_TICKER].loc[returns.index]
    
    # Panel 1: Precio con regimenes
    ax1 = axes[0]
    ax1.plot(price_data.index, price_data.values, color='black', linewidth=0.8, alpha=0.7)
    
    for i in range(len(states)):
        color = COLOR_CALMA if states[i] == 0 else COLOR_CRISIS
        alpha = 0.15 if states[i] == 0 else 0.25
        ax1.axvspan(price_data.index[i], 
                   price_data.index[min(i+1, len(states)-1)],
                   color=color, alpha=alpha)
    
    ax1.set_title('S&P 500 con Regimenes de Mercado Detectados', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Precio')
    legend_elements = [
        Patch(facecolor=COLOR_CALMA, alpha=0.3, label='Calma'),
        Patch(facecolor=COLOR_CRISIS, alpha=0.4, label='Crisis')
    ]
    ax1.legend(handles=legend_elements, loc='upper left')
    
    # Panel 2: Secuencia de estados
    ax2 = axes[1]
    ax2.fill_between(price_data.index, states, color=COLOR_CRISIS, alpha=0.6, step='post')
    ax2.set_ylabel('Estado')
    ax2.set_yticks([0, 1])
    ax2.set_yticklabels(['Calma', 'Crisis'])
    
    # Panel 3: Retornos
    ax3 = axes[2]
    colors_bar = [COLOR_CALMA if s == 0 else COLOR_CRISIS for s in states]
    ax3.bar(returns.index, returns[MARKET_TICKER].values * 100, color=colors_bar, alpha=0.6, width=1)
    ax3.axhline(y=0, color='black', linewidth=0.5)
    ax3.set_ylabel('Retorno (%)')
    ax3.set_xlabel('Fecha')
    
    plt.tight_layout()
    plt.savefig('../output/figures/regimenes_sp500.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_regimes(prices, states)


In [ ]:
# ==============================================================================
# ESTADISTICAS DE REGIMENES
# ==============================================================================

def regime_statistics(states):
    """
    Calcula estadisticas sobre los regimenes detectados.
    """
    total = len(states)
    n_calma = (states == 0).sum()
    n_crisis = (states == 1).sum()
    
    # Calcular duracion de periodos
    def get_period_lengths(states, target):
        lengths = []
        count = 0
        for s in states:
            if s == target:
                count += 1
            elif count > 0:
                lengths.append(count)
                count = 0
        if count > 0:
            lengths.append(count)
        return lengths
    
    periods_calma = get_period_lengths(states, 0)
    periods_crisis = get_period_lengths(states, 1)
    
    print("ESTADISTICAS DE REGIMENES")
    print("=" * 50)
    print(f"Total de observaciones: {total} dias")
    print()
    print("Estado CALMA:")
    print(f"  - Dias totales: {n_calma} ({n_calma/total:.1%})")
    print(f"  - Numero de periodos: {len(periods_calma)}")
    print(f"  - Duracion media: {np.mean(periods_calma):.1f} dias")
    print(f"  - Duracion maxima: {max(periods_calma)} dias")
    print()
    print("Estado CRISIS:")
    print(f"  - Dias totales: {n_crisis} ({n_crisis/total:.1%})")
    print(f"  - Numero de periodos: {len(periods_crisis)}")
    print(f"  - Duracion media: {np.mean(periods_crisis):.1f} dias")
    print(f"  - Duracion maxima: {max(periods_crisis)} dias")

regime_statistics(states)


---
## 4. Analisis de distribuciones marginales por estado

En esta seccion se cuantifica como cambian las distribuciones individuales de cada activo 
dependiendo del regimen de mercado. Se calculan estadisticos (media, volatilidad, skewness, kurtosis) 
condicionados a cada estado para responder:

- Cuanto aumenta la volatilidad de cada activo en crisis?
- El oro (GLD) funciona como activo refugio?
- Como se comporta el High Yield (HYG) en crisis?


In [ ]:
# ==============================================================================
# ESTADISTICOS CONDICIONALES POR ESTADO
# ==============================================================================

def conditional_statistics(returns, states):
    """
    Calcula estadisticos para cada activo, separados por estado del mercado.
    """
    returns_data = returns.drop(columns=['State'], errors='ignore')
    
    returns_calma = returns_data[states == 0]
    returns_crisis = returns_data[states == 1]
    
    def calc_stats(df, name):
        return pd.DataFrame({
            'Estado': name,
            'Media anual (%)': df.mean() * 252 * 100,
            'Vol anual (%)': df.std() * np.sqrt(252) * 100,
            'Skewness': df.apply(skew),
            'Kurtosis': df.apply(kurtosis)
        })
    
    stats_calma = calc_stats(returns_calma, 'Calma')
    stats_crisis = calc_stats(returns_crisis, 'Crisis')
    
    return stats_calma, stats_crisis

stats_calma, stats_crisis = conditional_statistics(returns, states)

print("ESTADISTICOS EN ESTADO DE CALMA")
print("=" * 60)
display(stats_calma.round(2))

print()
print("ESTADISTICOS EN ESTADO DE CRISIS")
print("=" * 60)
display(stats_crisis.round(2))


In [ ]:
# ==============================================================================
# COMPARACION DE VOLATILIDADES ENTRE ESTADOS
# ==============================================================================

def compare_volatilities(stats_calma, stats_crisis):
    """
    Compara volatilidades entre estados y calcula el ratio de aumento.
    """
    comparison = pd.DataFrame({
        'Vol Calma (%)': stats_calma['Vol anual (%)'],
        'Vol Crisis (%)': stats_crisis['Vol anual (%)'],
        'Ratio': stats_crisis['Vol anual (%)'] / stats_calma['Vol anual (%)'],
        'Aumento (%)': (stats_crisis['Vol anual (%)'] / stats_calma['Vol anual (%)'] - 1) * 100
    })
    return comparison.sort_values('Ratio', ascending=False).round(2)

vol_comparison = compare_volatilities(stats_calma, stats_crisis)

print("COMPARACION DE VOLATILIDADES: CALMA vs CRISIS")
print("=" * 60)
display(vol_comparison)

print(f"\nEn promedio, la volatilidad aumenta {vol_comparison['Ratio'].mean():.1f}x en crisis")


In [ ]:
# ==============================================================================
# VISUALIZACION: DISTRIBUCION DE RETORNOS POR ESTADO
# ==============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
assets_to_plot = ['GLD', 'HYG', MARKET_TICKER, 'NVDA', 'BAC', 'IEF']
assets_to_plot = [a for a in assets_to_plot if a in returns.columns]

returns_calma = returns[states == 0]
returns_crisis = returns[states == 1]

for idx, asset in enumerate(assets_to_plot):
    ax = axes[idx // 3, idx % 3]
    ax.hist(returns_calma[asset]*100, bins=50, alpha=0.6, 
            color=COLOR_CALMA, label='Calma', density=True)
    ax.hist(returns_crisis[asset]*100, bins=50, alpha=0.6, 
            color=COLOR_CRISIS, label='Crisis', density=True)
    ax.set_title(asset, fontweight='bold')
    ax.set_xlabel('Retorno (%)')
    if idx == 0:
        ax.legend()

plt.suptitle('Distribucion de Retornos por Estado del Mercado', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../output/figures/distribuciones_por_estado.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# ==============================================================================
# ANALISIS: ORO (GLD) COMO ACTIVO REFUGIO
# ==============================================================================

print("ANALISIS: ORO (GLD) COMO ACTIVO REFUGIO")
print("=" * 60)

returns_calma = returns[states == 0].drop(columns=['State'], errors='ignore')
returns_crisis = returns[states == 1].drop(columns=['State'], errors='ignore')

# Correlacion con el mercado
corr_calma = returns_calma['GLD'].corr(returns_calma[MARKET_TICKER])
corr_crisis = returns_crisis['GLD'].corr(returns_crisis[MARKET_TICKER])

print(f"Correlacion GLD vs S&P 500:")
print(f"  - En calma:  {corr_calma:.3f}")
print(f"  - En crisis: {corr_crisis:.3f}")

# Retorno medio
ret_calma = returns_calma['GLD'].mean() * 252 * 100
ret_crisis = returns_crisis['GLD'].mean() * 252 * 100

print(f"\nRetorno anualizado del oro:")
print(f"  - En calma:  {ret_calma:.2f}%")
print(f"  - En crisis: {ret_crisis:.2f}%")

if corr_crisis < corr_calma:
    print("\nConclusion: El oro muestra menor correlacion con el mercado en crisis,")
    print("            lo que sugiere un comportamiento de activo refugio.")


In [ ]:
# ==============================================================================
# ANALISIS: HIGH YIELD (HYG) EN CRISIS
# ==============================================================================

print("ANALISIS: HIGH YIELD (HYG) EN CRISIS")
print("=" * 60)

vol_calma_hyg = returns_calma['HYG'].std() * np.sqrt(252) * 100
vol_crisis_hyg = returns_crisis['HYG'].std() * np.sqrt(252) * 100

print(f"Volatilidad anualizada del High Yield:")
print(f"  - En calma:  {vol_calma_hyg:.2f}%")
print(f"  - En crisis: {vol_crisis_hyg:.2f}%")
print(f"  - Ratio:     {vol_crisis_hyg/vol_calma_hyg:.2f}x")

corr_calma_hyg = returns_calma['HYG'].corr(returns_calma[MARKET_TICKER])
corr_crisis_hyg = returns_crisis['HYG'].corr(returns_crisis[MARKET_TICKER])

print(f"\nCorrelacion HYG vs S&P 500:")
print(f"  - En calma:  {corr_calma_hyg:.3f}")
print(f"  - En crisis: {corr_crisis_hyg:.3f}")

print("\nConclusion: El High Yield se comporta mas como renta variable que como")
print("            renta fija en crisis, con mayor volatilidad y correlacion.")


In [ ]:
# ==============================================================================
# GRAFICO: RATIO DE VOLATILIDAD POR ACTIVO
# ==============================================================================

fig, ax = plt.subplots(figsize=(10, 8))

data = vol_comparison.sort_values('Ratio')
avg_ratio = data['Ratio'].mean()

colors = [COLOR_CRISIS if r > avg_ratio else COLOR_CALMA for r in data['Ratio']]
bars = ax.barh(data.index, data['Ratio'], color=colors, alpha=0.7)

ax.axvline(x=1, color='black', linewidth=1, label='Sin cambio')
ax.axvline(x=avg_ratio, color='gray', linestyle='--', label=f'Media ({avg_ratio:.1f}x)')

ax.set_xlabel('Ratio de Volatilidad (Crisis / Calma)')
ax.set_title('Aumento de Volatilidad en Crisis por Activo', fontweight='bold')
ax.legend(loc='lower right')

for bar, val in zip(bars, data['Ratio']):
    ax.text(val + 0.05, bar.get_y() + bar.get_height()/2, f'{val:.1f}x', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../output/figures/volatility_ratios.png', dpi=150, bbox_inches='tight')
plt.show()


---
## 5. Estructura de dependencia (Copulas)

En esta seccion se analiza como cambia la estructura de dependencia entre activos 
segun el regimen de mercado. El objetivo es demostrar que en crisis las correlaciones 
tienden a aumentar (la diversificacion "falla").


In [ ]:
# TODO: Implementar ajuste de copulas
# - Matriz de correlacion en estado Calma
# - Matriz de correlacion en estado Crisis
# - Ajuste de copulas (Gaussiana, t-Student, Clayton)
# - Comparacion de parametros de dependencia

print("Seccion pendiente de implementacion")


---
## 6. Motor de simulacion Monte Carlo

Simulacion de 10,000 trayectorias a 6 meses que respeten:
- La dinamica de regimenes (matriz de transicion del HMM)
- Las distribuciones marginales de cada estado
- La estructura de dependencia (copulas) de cada estado


In [ ]:
# TODO: Implementar motor de simulacion
# - Simulador de cadena de Markov
# - Generacion de retornos correlacionados por estado
# - Construccion de trayectorias de cartera
# - Validacion: comparacion real vs simulado

print("Seccion pendiente de implementacion")


---
## 7. Escenarios de estres

Construccion de escenarios adversos para "romper la cartera":
- **Escenario 1 (Estanflacion 2022):** Alta inflacion + bajo crecimiento
- **Escenario 2 (Crisis de Credito 2008):** Colapso del sistema crediticio
- **Escenario 3:** Escenario alternativo con justificacion economica


In [ ]:
# TODO: Implementar escenarios de estres
# - Escenario Estanflacion 2022
# - Escenario Crisis Crediticia 2008
# - Escenario alternativo
# - Calculo de VaR 99% y CVaR bajo cada escenario

print("Seccion pendiente de implementacion")


---
## 8. Conclusiones

### Hallazgos principales

[Por completar tras el analisis]

### Recomendaciones para el Comite de Riesgos

[Por completar]
